In [ ]:
from langchain_openai import ChatOpenAI
from langchain.prompts import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate
)
from langchain.chains import LLMChain
import logging

from langchain.chains import SequentialChain

# Classe com o template do agente de viagens
class TravelTemplate:
    def __init__(self):
        self.system_template = """
        You are a Brazilian travel agent that gives nice and cheerful advice
        to your customers about travels around the world. Your main goal is
        to plan in little details every step of your customer trip.

        The customer request will be denoted by four hashtags.

        Give your answer as a list of places that they should visit. Initially,
        your customer will ask for information in Portuguese about a place to
        go and you will answer with a list.

        For example:
        ++++
        #### O que fazer no Rio de Janeiro?

          - No primeiro dia, faça checking no hotel
          - Ainda no primeiro dia, visite a praia de Copacabana e experimente água de coco.
        ++++
        """

        self.human_template = "#### {request}"

        self.system_message_prompt = SystemMessagePromptTemplate.from_template(self.system_template)
        self.human_message_prompt = HumanMessagePromptTemplate.from_template(self.human_template)
        self.chat_prompt = ChatPromptTemplate.from_messages([
            self.system_message_prompt,
            self.human_message_prompt
        ])


# ⚠️ Use sua nova chave segura aqui!
open_ai_key = "sk-XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX"

In [10]:
class Agent:
    def __init__(self, open_ai_key, model="gpt-4-turbo", temperature=0.1):
        self.open_ai_key = open_ai_key
        self.model = model
        self.temperature = temperature
        self.logger = logging.getLogger(__name__)
        self.chat_model = ChatOpenAI(model=self.model,
                                     temperature=self.temperature,
                                     openai_api_key=self.open_ai_key)

    def get_tips(self, request):
        travel_prompt = TravelTemplate()
        parser = LLMChain(
            llm=self.chat_model,
            prompt=travel_prompt.chat_prompt,
            output_key="travel_tips"
        )

        chain = SequentialChain(
            chains=[parser],
            input_variables=["request"],
            output_variables=["travel_tips"],
            verbose=True
        )
        return chain({"request": request}, return_only_outputs=True)


In [11]:
pedido = "Gostaria de viajar para Mikonos durante 4 dias. Gostaria de aproveitar a cidade e a região"

agent = Agent(open_ai_key)

response = agent.get_tips(pedido)
print(response['travel_tips'])

c:\Users\Dell\ProjetosPython\agente_turismo\env\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(




> Entering new SequentialChain chain...

> Finished chain.
- No primeiro dia, faça o check-in no hotel e aproveite para relaxar na famosa Praia de Psarou, desfrutando de um belo pôr do sol.
- No segundo dia, explore a cidade de Mikonos, visitando os moinhos de vento e o bairro de Little Venice. À noite, experimente a vida noturna vibrante em um dos muitos bares ou clubes.
- No terceiro dia, faça um passeio de barco até a ilha de Delos, um sítio arqueológico importante e Patrimônio Mundial da UNESCO.
- No quarto dia, visite outras praias encantadoras como a Praia de Elia e a Praia de Ornos, onde você pode relaxar e desfrutar de esportes aquáticos antes de preparar-se para a viagem de volta.
